<a href="https://colab.research.google.com/github/CHO-MINJUNG/BigContest2020/blob/master/bigcon_test_model_mm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
teamhitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/3차_최종_최근_상대_팀타자.csv', encoding='utf-8-sig')

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os
from pathlib import Path

folder = "bigcon2020/data"
base_path = Path("/content/drive/My Drive/")
project_path = base_path / folder
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"현재 디렉토리 위치: {os.getcwd()}")

FileNotFoundError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

print('tensorflow version: {}'.format(tf.__version__))
print('GPU 사용 가능 여부: {}'.format(tf.test.is_gpu_available()))

tensorflow version: 2.3.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU 사용 가능 여부: True


In [ ]:
batch_size = 9
max_epochs = 30
learning_rate = 4e-5
hidden_sizes = [128, 128]

In [ ]:
def preprocess():
  # 1차_최종_공통 : 시즌, 홈어웨이, 경기간간격, 득점, 실점, 승패
  general = pd.read_csv('1차_최종_공통.csv')
  general = general[general['팀코드']=='NC']
  general = general[['게임키', '경기간간격','득점','실점','결과']]
  general['시즌'] = general['게임키'].apply(lambda x:int(x[:4]))
  general['홈어웨이'] = general['게임키'].apply(lambda x:0 if x[-3:-1]=='NC' else 1)
  general=general.set_index('게임키')
  # 팀타자 : 기준팀 wRC
  hitter = pd.read_csv('./1차_최종_팀타자.csv')
  hitter_vs = hitter[hitter['상대팀코드']=='NC']
  hitter = hitter[hitter['팀코드']=='NC']
  hitter = hitter[['게임키', 'wRC']]
  hitter_vs = hitter_vs[['게임키', 'wRC']]
  hitter=hitter.set_index('게임키')
  hitter_vs=hitter_vs.set_index('게임키')
  hitter_vs=hitter_vs.add_prefix('vs_')
  # 팀투수 : 기준팀, 상대팀 RA/9, whip
  pitcher = pd.read_csv('./1차_최종_팀투수.csv')
  pitcher_vs = pitcher[pitcher['상대팀코드']=='NC']
  pitcher = pitcher[pitcher['팀코드']=='NC']
  pitcher = pitcher[['게임키', 'RA/9','whip']]
  pitcher_vs = pitcher_vs[['게임키', 'RA/9','whip']]
  pitcher = pitcher.set_index('게임키')
  pitcher_vs = pitcher_vs.set_index('게임키')
  pitcher_vs = pitcher_vs.add_prefix('vs_')
  df = pd.concat([general, hitter,hitter_vs, pitcher,pitcher_vs], axis=1)
  df= df[['시즌','경기간간격','홈어웨이','wRC','vs_wRC','RA/9','whip','vs_RA/9','vs_whip','득점','실점','결과']]
  df['결과'] = df['결과'].apply(lambda x : 0 if x =='W' else 1)
  return df

In [ ]:
df = preprocess()
df.head()

,시즌,경기간간격,홈어웨이,wRC,vs_wRC,RA/9,whip,vs_RA/9,vs_whip,득점,실점,결과
게임키,,,,,,,,,,,,
20160401HTNC0,2016,0.0,0,0.000,0.000,0.000,0.000,0.000,0.000,5,4,0
20160402HTNC0,2016,1.0,0,7.039,4.188,4.000,1.222,5.625,1.750,3,4,1
20160405NCOB0,2016,3.0,1,9.400,12.440,4.000,1.333,5.824,1.647,2,6,1
20160406NCOB0,2016,1.0,1,10.757,18.658,4.846,1.423,4.500,1.462,0,2,1
20160407NCOB0,2016,1.0,1,7.610,19.111,4.235,1.265,3.343,1.171,8,2,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 639 entries, 20160401HTNC0 to 20200719KTNC0
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   시즌       639 non-null    int64  
 1   경기간간격    639 non-null    float64
 2   홈어웨이     639 non-null    int64  
 3   wRC      639 non-null    float64
 4   vs_wRC   639 non-null    float64
 5   RA/9     639 non-null    float64
 6   whip     639 non-null    float64
 7   vs_RA/9  639 non-null    float64
 8   vs_whip  639 non-null    float64
 9   득점       639 non-null    int64  
 10  실점       639 non-null    int64  
 11  결과       639 non-null    int64  
dtypes: float64(7), int64(5)
memory usage: 64.9+ KB


In [ ]:
def normalization(df, columns, mode='z'):
  if mode == 'm' :
    for colname in columns:
      df[colname] = (df[colname] - df[colname].min())/(df[colname].max()-df[colname].min())
    return df
  if mode == 'z' :
    for colname in columns:
      df[colname] = (df[colname] - df[colname].mean())/df[colname].std()
    return df

In [ ]:
# df = normalization(df,['경기간간격', 'wRC', 'vs_wRC', 'RA/9', 'whip', 'vs_RA/9', 'vs_whip', '득점', '실점'],mode ="m")
df = normalization(df,['경기간간격', 'wRC', 'vs_wRC', 'RA/9', 'whip', 'vs_RA/9', 'vs_whip', '득점', '실점'],mode ="z")

In [ ]:
train_data= np.asarray(df[(df['시즌']!= 2019) & (df['시즌']!= 2020)].drop('시즌', axis=1)).astype(np.float32)
val_data= np.asarray(df[df['시즌']==2019].drop('시즌', axis=1)).astype(np.float32)
test_data= np.asarray(df[df['시즌']==2020].drop('시즌', axis=1)).astype(np.float32)

In [ ]:
print("shape of train data: {}".format(train_data.shape))
print("shape of val data: {}".format(val_data.shape))
print("shape of test data: {}".format(test_data.shape))

shape of train data: (432, 11)
shape of val data: (144, 11)
shape of test data: (63, 11)


In [ ]:
def make_dataset(data, seq_length=5, target_delay=0, strides=1):
    sequence = []
    target = []
    for index in range(len(data) - seq_length - target_delay):
        if index % strides == 0:
          sequence.append(data[index:index+seq_length])
          target.append(data[index+seq_length+target_delay][-1])
    return np.array(sequence), np.array(target)

In [ ]:
train_sequences, train_labels = make_dataset(train_data)
val_sequences, val_labels = make_dataset(val_data)
test_sequences, test_labels = make_dataset(test_data)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sequences, train_labels))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.repeat()

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_sequences, val_labels))
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.repeat()

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_sequences, test_labels))
test_dataset = test_dataset.batch(batch_size)

In [ ]:
model = tf.keras.Sequential()

In [ ]:
num_layers = len(hidden_sizes)
for i in range(num_layers - 1):
    model.add(tf.keras.layers.LSTM(128,return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001),dropout=0.2))
    model.add(tf.keras.layers.LSTM(64,return_sequences=False,kernel_regularizer=tf.keras.regularizers.l2(0.001),dropout=0.2))
    model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01) ,loss='binary_crossentropy',metrics=['binary_accuracy'])

In [ ]:
loss, acc = model.evaluate(test_sequences, test_labels)
print("학습 전 모델의 loss: {:.5f}".format(loss))
print("학습 전 모델의 정확도: {:5.2f}%".format(100*acc))

2/2 [==============================] - 0s 3ms/step - loss: 0.8850 - binary_accuracy: 0.4655
학습 전 모델의 loss: 0.88504
학습 전 모델의 정확도: 46.55%


In [ ]:
checkpoint_path = './checkpoint_z_score/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=1
                            )

In [ ]:
model.fit(train_dataset, steps_per_epoch = train_data.shape[1]/batch_size, 
          validation_data = val_dataset, 
          validation_steps= val_data.shape[1]/batch_size,
          epochs= max_epochs,
          callbacks= [checkpoint])

Epoch 1/30
1/1 [=======================>......] - ETA: 0s - loss: 0.8801 - binary_accuracy: 0.5556
Epoch 00001: val_loss improved from inf to 0.89354, saving model to ./checkpoint_z_score/cp.ckpt
2/1 [=================================================] - 0s 30ms/step - loss: 0.8830 - binary_accuracy: 0.5000 - val_loss: 0.8935 - val_binary_accuracy: 0.4444
Epoch 2/30
1/1 [=======================>......] - ETA: 0s - loss: 0.8575 - binary_accuracy: 0.8889
Epoch 00002: val_loss improved from 0.89354 to 0.89288, saving model to ./checkpoint_z_score/cp.ckpt
2/1 [=================================================] - 0s 36ms/step - loss: 0.8651 - binary_accuracy: 0.7778 - val_loss: 0.8929 - val_binary_accuracy: 0.3889
Epoch 3/30
1/1 [=======================>......] - ETA: 0s - loss: 0.8740 - binary_accuracy: 0.6667
Epoch 00003: val_loss improved from 0.89288 to 0.89184, saving model to ./checkpoint_z_score/cp.ckpt
2/1 [=================================================] - 0s 35ms/step - loss: 0.8

In [ ]:
for sequences, targets in train_dataset.take(1):
    outputs = model(sequences)
    print(outputs)

tf.Tensor(
[[0.4519733 ]
 [0.48965803]
 [0.48364684]
 [0.4783963 ]
 [0.4775293 ]
 [0.4925512 ]
 [0.49176925]
 [0.49237034]
 [0.4868963 ]], shape=(9, 1), dtype=float32)


In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
_, test_loss = model.evaluate(test_dataset, steps=len(test_sequences)/batch_size)

7/6 [================================] - 0s 3ms/step - loss: 0.8771 - binary_accuracy: 0.6034


In [ ]:
loss, acc = model.evaluate(test_sequences, test_labels)
print("학습 후 모델의 loss: {:.5f}".format(loss))
print("학습 후 모델의 정확도: {:5.2f}%".format(100*acc))

2/2 [==============================] - 0s 4ms/step - loss: 0.8771 - binary_accuracy: 0.6034
학습 후 모델의 loss: 0.87711
학습 후 모델의 정확도: 60.34%
